In [43]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN


In [2]:
fp = Path('Resources/Updated/play_prob1.csv')

df_2 = pd.read_csv(fp)

df_2 = pd.DataFrame(df_2)

df_2

,Unnamed: 0,posteam,play_type,down,ydstogo,qtr,week,yardline_100,drive_inside20,yards_gained,success,series_result,fg_prob,td_prob,no_score_prob,safety_prob,opp_td_prob,wp
0,0,TEN,run,1.0,10,1,1,75,0.0,-3,0,Punt,0.212601,0.406750,0.004661,0.003828,0.226051,0.546262
1,1,TEN,pass,2.0,13,1,1,78,0.0,3,1,Punt,0.167558,0.321729,0.004958,0.003786,0.296200,0.520599
2,2,TEN,pass,3.0,10,1,1,75,0.0,0,0,Punt,0.178285,0.319862,0.005425,0.003580,0.299509,0.498319
3,3,ARI,pass,1.0,10,1,1,61,1.0,38,1,First down,0.257016,0.404707,0.005188,0.004175,0.213975,0.522434
4,4,ARI,run,1.0,10,1,1,23,1.0,2,0,First down,0.391608,0.529298,0.002235,0.000826,0.047409,0.616783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,35761,CIN,pass,1.0,10,4,22,75,0.0,17,1,First down,0.233165,0.161310,0.451072,0.001453,0.058733,0.312859
35762,35762,CIN,pass,1.0,10,4,22,58,0.0,9,1,Turnover on downs,0.268742,0.209430,0.418316,0.001627,0.046851,0.365383
35763,35763,CIN,pass,2.0,1,4,22,49,0.0,0,0,Turnover on downs,0.312642,0.235814,0.383152,0.000715,0.030652,0.406260
35764,35764,CIN,run,3.0,1,4,22,49,0.0,0,0,Turnover on downs,0.287262,0.189174,0.445391,0.000627,0.035990,0.331002


In [3]:
df_2['play_type'] = df_2['play_type'].astype(str).str.replace('run' , '1')
df_2['play_type'] = df_2['play_type'].astype(str).str.replace('pass' , '0')
df_2['play_type'] = df_2['play_type'].astype('float')
df_2


,Unnamed: 0,posteam,play_type,down,ydstogo,qtr,week,yardline_100,drive_inside20,yards_gained,success,series_result,fg_prob,td_prob,no_score_prob,safety_prob,opp_td_prob,wp
0,0,TEN,1.0,1.0,10,1,1,75,0.0,-3,0,Punt,0.212601,0.406750,0.004661,0.003828,0.226051,0.546262
1,1,TEN,0.0,2.0,13,1,1,78,0.0,3,1,Punt,0.167558,0.321729,0.004958,0.003786,0.296200,0.520599
2,2,TEN,0.0,3.0,10,1,1,75,0.0,0,0,Punt,0.178285,0.319862,0.005425,0.003580,0.299509,0.498319
3,3,ARI,0.0,1.0,10,1,1,61,1.0,38,1,First down,0.257016,0.404707,0.005188,0.004175,0.213975,0.522434
4,4,ARI,1.0,1.0,10,1,1,23,1.0,2,0,First down,0.391608,0.529298,0.002235,0.000826,0.047409,0.616783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,35761,CIN,0.0,1.0,10,4,22,75,0.0,17,1,First down,0.233165,0.161310,0.451072,0.001453,0.058733,0.312859
35762,35762,CIN,0.0,1.0,10,4,22,58,0.0,9,1,Turnover on downs,0.268742,0.209430,0.418316,0.001627,0.046851,0.365383
35763,35763,CIN,0.0,2.0,1,4,22,49,0.0,0,0,Turnover on downs,0.312642,0.235814,0.383152,0.000715,0.030652,0.406260
35764,35764,CIN,1.0,3.0,1,4,22,49,0.0,0,0,Turnover on downs,0.287262,0.189174,0.445391,0.000627,0.035990,0.331002


In [4]:

print(df_2.columns.tolist())


['Unnamed: 0', 'posteam', 'play_type', 'down', 'ydstogo', 'qtr', 'week', 'yardline_100', 'drive_inside20', 'yards_gained', 'success', 'series_result', 'fg_prob', 'td_prob', 'no_score_prob', 'safety_prob', 'opp_td_prob', 'wp']


In [10]:

df_2 = df_2[['success' , 'yards_gained' , 'down' , 'ydstogo' , 'play_type']].convert_dtypes()


In [11]:

df_2 = df_2[df_2.down != 0 ]



In [12]:
df_2['down'].isnull().sum()


0

In [13]:

df_2 = df_2[['down' , 'play_type' , 'ydstogo' , 'success' , 'yards_gained']].convert_dtypes()
df_2


,down,play_type,ydstogo,success,yards_gained
0,1,1,10,0,-3
1,2,0,13,1,3
2,3,0,10,0,0
3,1,0,10,1,38
4,1,1,10,0,2
...,...,...,...,...,...
35761,1,0,10,1,17
35762,1,0,10,1,9
35763,2,0,1,0,0
35764,3,1,1,0,0


In [14]:
df_2 = df_2.dropna(axis = 'columns' , how='any')

df_2 = df_2.dropna()

df_2


,down,play_type,ydstogo,success,yards_gained
0,1,1,10,0,-3
1,2,0,13,1,3
2,3,0,10,0,0
3,1,0,10,1,38
4,1,1,10,0,2
...,...,...,...,...,...
35761,1,0,10,1,17
35762,1,0,10,1,9
35763,2,0,1,0,0
35764,3,1,1,0,0


In [15]:

x_2 = {'down_1': '1' , 'down_2': '2' , 'down_3': '3' , 'down_4': '4'}
df_2 = df_2.replace(x_2)


df_2.reset_index(inplace=True, drop=True)

df_2.head(10)


,down,play_type,ydstogo,success,yards_gained
0,1,1,10,0,-3
1,2,0,13,1,3
2,3,0,10,0,0
3,1,0,10,1,38
4,1,1,10,0,2
5,1,0,18,0,1
6,2,0,17,1,21
7,1,1,9,0,0
8,2,0,24,0,8
9,3,0,16,0,0


In [16]:


X_2 = df_2.drop('play_type' , axis=1)

X_2 = pd.get_dummies(X_2)

# Create our target
y_2 = df_2['play_type'].astype('int')


In [17]:
X_2.describe()


,down,ydstogo,success,yards_gained
count,35608.000000,35608.000000,35608.000000,35608.000000
mean,1.809958,8.439845,0.442148,5.515867
std,0.837031,3.973347,0.496649,8.518125
min,1.000000,1.000000,0.000000,-28.000000
25%,1.000000,6.000000,0.000000,0.000000
50%,2.000000,10.000000,0.000000,3.000000
75%,2.000000,10.000000,1.000000,8.000000
max,4.000000,43.000000,1.000000,91.000000


In [18]:
y_2.value_counts()


0    20898
1    14710
Name: play_type, dtype: int64

In [70]:

X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, random_state=1)



In [71]:
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_2_train, y_2_train)
y_2_pred = brfc.predict(X_2_test)
y_2_pred = brfc.predict(X_2_test)


In [72]:

balanced_accuracy_score(y_2_test , y_2_pred)
balanced_accuracy_score(y_2_test , y_2_pred)
confusion_matrix(y_2_test, y_2_pred)
print(classification_report_imbalanced(y_2_test, y_2_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.80      0.74      0.73      0.77      0.74      0.55      5233
          1       0.67      0.73      0.74      0.70      0.74      0.54      3669

avg / total       0.74      0.74      0.74      0.74      0.74      0.55      8902



In [30]:

results_2 = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)
results_2.head(20)


,Prediction,Actual
0,0,0
1,1,0
2,0,0
3,0,0
4,1,1
5,1,1
6,1,0
7,1,0
8,0,0
9,0,1


In [45]:
results_2 = pd.DataFrame({"Prediction": y_2_pred})
results_2

,Prediction
0,0
1,0
2,0
3,0
4,1
...,...
8897,1
8898,1
8899,1
8900,1


In [46]:
results_3 = pd.DataFrame({"Actual": y_2_test})
results_3

,Actual
23090,0
15992,0
18214,0
33453,0
4323,1
...,...
2069,1
17764,1
25623,1
11955,1


In [51]:
a = accuracy_score(results_2['Prediction'], results_3['Actual'])
    
print(f'accuracy: {a*100:.2f}%')

accuracy: 74.57%


In [42]:
hello = confusion_matrix(y_2_test, y_2_pred)

table = pd.DataFrame(hello , index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])

table

,Actual,Predicted
Pass,3871,1362
Run,902,2767


In [35]:
eec = EasyEnsembleClassifier(n_estimators=100 , random_state=1)
eec = eec.fit(X_2_train , y_2_train)
y_2_pred = eec.predict(X_2_test)


In [36]:

balanced_accuracy_score(y_2_test , y_2_pred)

confusion_matrix(y_2_test , y_2_pred)
print(classification_report_imbalanced(y_2_test , y_2_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.81      0.74      0.75      0.77      0.75      0.56      5233
          1       0.67      0.75      0.74      0.71      0.75      0.56      3669

avg / total       0.75      0.75      0.75      0.75      0.75      0.56      8902



In [52]:

ecc_results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)
ecc_results.head(20)


,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,1,1
5,1,1
6,0,0
7,1,0
8,0,0
9,0,1


In [54]:
ecc_results_1 = pd.DataFrame({"Prediction": y_2_pred})
ecc_results_1.head(5)


,Prediction
0,0
1,0
2,0
3,0
4,1


In [55]:
ecc_results_2 = pd.DataFrame({"Actual": y_2_test})
ecc_results_2.head(5)


,Actual
23090,0
15992,0
18214,0
33453,0
4323,1


In [56]:
a_2 = accuracy_score(ecc_results_1['Prediction'], ecc_results_2['Actual'])
    
print(f'accuracy: {a_2*100:.2f}%')

accuracy: 74.57%


In [38]:
hello_2 = confusion_matrix(y_2_test, y_2_pred)

table_2 = pd.DataFrame(hello , index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])

table_2

,Actual,Predicted
Pass,3888,1345
Run,976,2693


In [57]:
ros = RandomOverSampler(random_state=1)
X_2_resamp, y_2_resamp = ros.fit_resample(X_2_train, y_2_train)
Counter(y_2_resamp)


Counter({0: 15665, 1: 15665})

In [58]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_2_resamp, y_2_resamp)


LogisticRegression(random_state=1)

In [62]:
y_2_pred = model.predict(X_2_test)
balanced_accuracy_score(y_2_test, y_2_pred)

print(classification_report_imbalanced(y_2_test , y_2_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.55      0.74      0.64      0.64      0.40      5233
          1       0.54      0.74      0.55      0.62      0.64      0.42      3669

avg / total       0.66      0.63      0.66      0.63      0.64      0.41      8902



In [60]:
ros_results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)
ros_results.head(20)


,Prediction,Actual
0,0,0
1,1,0
2,1,0
3,0,0
4,1,1
5,1,1
6,0,0
7,1,0
8,0,0
9,0,1


In [63]:
ros_results_1 = pd.DataFrame({"Prediction": y_2_pred})
ros_results_1.head(5)


,Prediction
0,0
1,1
2,1
3,0
4,1


In [64]:
ros_results_2 = pd.DataFrame({"Actual": y_2_test})
ros_results_2.head(5)


,Actual
23090,0
15992,0
18214,0
33453,0
4323,1


In [66]:
a_3 = accuracy_score(ros_results_1['Prediction'], ros_results_2['Actual'])
    
print(f'accuracy: {a_3*100:.2f}%')

accuracy: 63.01%


In [67]:
ros_matrix = confusion_matrix(y_2_test, y_2_pred)
ros_df = pd.DataFrame(
    ros_matrix, index = ['Pass' , 'Run'] , columns=['Actual' , 'Predicted'])
ros_df


,Actual,Predicted
Pass,2890,2343
Run,950,2719


#### Smote Oversampling 


In [74]:
X_2_train = X_2_train.astype('float') 

y_2_train = y_2_train.astype('float')

X_2_resamp, y_2_resamp = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(X_2_train, y_2_train)


In [75]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_2_resamp, y_2_resamp)


LogisticRegression(random_state=1)

In [111]:
y_2_pred_s = model.predict(X_2_test)
balanced_accuracy_score(y_2_test, y_2_pred_s)



0.6477030295558664

In [107]:
print(classification_report_imbalanced(y_2_test, y_2_pred_s))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.55      0.74      0.64      0.64      0.40      5233
          1       0.54      0.74      0.55      0.62      0.64      0.42      3669

avg / total       0.66      0.63      0.66      0.63      0.64      0.41      8902



In [108]:
smot_results_1 = pd.DataFrame({"Prediction": y_2_pred_s})
smot_results_1.head(5)


,Prediction
0,0.00%
1,100.00%
2,100.00%
3,0.00%
4,100.00%


In [109]:
smot_results_2 = pd.DataFrame({"Actual": y_2_test})
smot_results_2.head(5)


,Actual
23090,0
15992,0
18214,0
33453,0
4323,1


In [120]:
a_4 = accuracy_score(smot_results_1['Prediction'], smot_results_2['Actual'])
    
print(f'accuracy: {a_4*100:.2f}%')

accuracy: 63.06%


In [86]:
matrix_sm = confusion_matrix(y_2_test, y_2_pred_s)

smot_results = pd.DataFrame(matrix_sm, index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])

smot_results


,Actual,Predicted
Pass,2886,2347
Run,941,2728



#### Undersampling


In [97]:
X_2_train = X_2_train.astype('int') 

y_2_train = y_2_train.astype('int')


In [88]:
cc = ClusterCentroids(random_state=1)
X_2_resample3, y_2_resample3 = cc.fit_resample(X_2_train, y_2_train)
Counter(y_2_resample3)


Counter({0: 11041, 1: 11041})

In [89]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_2_resample3, y_2_resample3)
y_2_pred_cc = model.predict(X_2_test)


In [119]:
balanced_accuracy_score(y_2_test, y_2_pred_cc)


0.6477030295558664

In [116]:
cc_results_1 = pd.DataFrame({"Prediction": y_2_pred_cc})
cc_results_1.head(5)


,Prediction
0,0
1,1
2,1
3,0
4,1


In [117]:
cc_results_2 = pd.DataFrame({"Actual": y_2_test})
cc_results_2.head(5)


,Actual
23090,0
15992,0
18214,0
33453,0
4323,1


In [123]:
a_5 = accuracy_score(cc_results_1['Prediction'], cc_results_2['Actual'])
    
print(f'accuracy: {a_5*100:.2f}%')

accuracy: 63.05%


In [124]:
matrix_cc = confusion_matrix(y_2_test, y_2_pred_cc)


In [125]:
cc_results = pd.DataFrame(
    matrix_cc, index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])
cc_results


,Actual,Predicted
Pass,2878,2355
Run,934,2735


In [126]:
print(classification_report_imbalanced(y_2_test, y_2_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.55      0.75      0.64      0.64      0.40      5233
          1       0.54      0.75      0.55      0.62      0.64      0.42      3669

avg / total       0.67      0.63      0.66      0.63      0.64      0.41      8902



#### Under and over sampling 

In [131]:
y_2 = y_2.astype('float')

X_2 = X_2.astype('float')


In [132]:
smote = SMOTEENN(random_state=0)
X_resamp_teen, y_resamp_teen = smote.fit_resample(X_2, y_2)
Counter(y_resamp_teen)

Counter({0.0: 12179, 1.0: 4287})

In [133]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resamp_teen, y_resamp_teen)

LogisticRegression(random_state=1)

In [134]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_2_test, y_2_pred)

0.6475134710498406

In [135]:
# Display the confusion matrix
y_pred = model.predict(X_2_test)
confusion_matrix(y_2_test, y_2_pred)

array([[2886, 2347],
       [ 941, 2728]], dtype=int64)

In [136]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_2_test, y_2_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.55      0.74      0.64      0.64      0.40      5233
          1       0.54      0.74      0.55      0.62      0.64      0.42      3669

avg / total       0.66      0.63      0.66      0.63      0.64      0.41      8902

